### This notebook is to generate df of Positions, that is resulted in our strategy

In [1]:
import pandas as pd
from math import sqrt
import math

import requests
import json
import numpy as np

import UNI_v3_funcs as liq_amounts
import load_data

### Query Functions

In [2]:
def get_cprice_at_mint(token0, token1, feeTier, mint_time):
    
    # get token_ids
    token0_id = load_data.get_token_id(token0)
    token1_id = load_data.get_token_id(token1)
    pool_id = load_data.get_pool_id(token0_id, token1_id, feeTier)
        
    # query price at that specific time
    query_ = """
    {{
      poolHourDatas(first:1,
      where:{{ pool: "{}",
      periodStartUnix: {} }}
      ){{
        tick
      }}
    }}""".format(pool_id, mint_time)
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    ctick = int(json_data_['data']['poolHourDatas'][0]['tick'])
    cprice = 1.0001**ctick
    
    return cprice

def get_decimals(token_symbol):
    query_ = """
    {{
      tokens(first:10,
      where:{{ symbol: "{}"}}
      ){{
        decimals
      }}
    }}""".format(token_symbol)
    
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    decimal_ = float(json_data_['data']['tokens'][0]['decimals'])
    
    return decimal_

### Calculations Functions

In [3]:
def price_to_tick(price_):
    # convert price (token1 by token0) to tick
    tick_ = math.floor(math.log(sqrt(price_), math.sqrt(1.0001)))
    return tick_

def tick_to_price(tick_):
    price_ = 1.0001**tick_
    return price_

In [19]:
# L_you calculation / inRange
# 4B. JNP: Use 'get_liquidity' function to calculate liquidity as a function of amounts and price range
def get_liquidity0(sqrtA,sqrtB,amount0,decimals):
    
    if (sqrtA > sqrtB):
          (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity=amount0/((2**96*(sqrtB-sqrtA)/sqrtB/sqrtA)/10**decimals)
    return liquidity

def get_liquidity1(sqrtA,sqrtB,amount1,decimals):
    
    if (sqrtA > sqrtB):
        (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity=amount1/((sqrtB-sqrtA)/2**96/10**decimals)
    return liquidity

def get_liquidity(tick,tickA,tickB,amount0,amount1,decimal0,decimal1):
    
        sqrt=(1.0001**(tick/2)*(2**96))
        sqrtA=(1.0001**(tickA/2)*(2**96))
        sqrtB=(1.0001**(tickB/2)*(2**96))
        if (sqrtA > sqrtB):
            (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
        if sqrt<=sqrtA:
            
            liquidity0=get_liquidity0(sqrtA,sqrtB,amount0,decimal0)
            inRange = False
            return liquidity0, inRange
        elif sqrt<sqrtB and sqrt>sqrtA:

            liquidity0=get_liquidity0(sqrt,sqrtB,amount0,decimal0)
            liquidity1=get_liquidity1(sqrtA,sqrt,amount1,decimal1)
            
            liquidity=liquidity0 if liquidity0<liquidity1 else liquidity1
            inRange = True
            return liquidity, inRange
        
        else:
            liquidity1=get_liquidity1(sqrtA,sqrtB,amount1,decimal1)
            inRange = False
            return liquidity1, inRange

In [27]:
# Calculate Fees 
def get_fees(lower_tick_, upper_tick_, ctick_,  L_Pool_, swap_Volume_, amt0_, amt1_, decimal0_, decimal1_, feeTier):
    # (lower_tick_, upper_tick_) from df_positions
    # (ctick, L_Pool, swap_volume) from merge
    # (amt0, amt1) - this changes based on swaps, but doesnt matter, bacause liquidity always constant
    # (decimal0, decimal1, feetier) - fixed
    
#     tick_ = price_to_tick(cprice_)
#     tickA_ = price_to_tick(lower_)
#     tickB_ = price_to_tick(upper_)
    L_you_, inRange_ = get_liquidity(ctick_, lower_tick_,upper_tick_, amt0_, amt1_, decimal0_, decimal1_)

    if inRange_:
        pool_share_ = (L_you_/L_Pool_)
        pool_fee_rate_ = float(feeTier) / 10000
        fees = pool_share_ * swap_Volume_ * pool_fee_rate_ / 100
    else:
        fees = 0
        
    return fees

In [6]:
# Step 4: use JNP's code
def get_impermanent_loss():
    return IL

In [7]:
# Step 4
def get_PNL():
    return pnl

In [8]:
# Step 5
def get_cost():
    return cost

### Positions Genrations

In [9]:
def get_positions_fixed_strategy(mint_time, burn_time, upper_, lower_):
    
    df_positions = pd.DataFrame()
    df_positions['periodStartUnix'] = list(range(mint_time, burn_time+1, 3600))
    df_positions['upper'] = upper_
    df_positions['lower'] = lower_
    df_positions['upper_tick'] = price_to_tick(upper_)
    df_positions['lower_tick'] = price_to_tick(lower_)
    
    return df_positions

### PNL Wrapper code

In [10]:
def calculate_pnl_from_positions(df_merged, df_positions):

    merge_feat = ['periodStartUnix', 'tick','liquidity','amountUSD']
    df_positions_fees = df_positions.merge(df_merged[merge_feat], how='left', on='periodStartUnix')
    df_positions_fees['liquidity'] = df_positions_fees['liquidity'].astype(float)

    df_positions_fees = df_positions_fees.reset_index() # cleaning it just in case
    for index, row in df_positions_fees.iterrows():
        row = row.copy()
        fees = get_fees(row['lower_tick'],row['upper_tick'],row['tick'],float(row['liquidity']),row['amountUSD'],
                                                MINT_AMT0, MINT_AMT1, DECIMAL0, DECIMAL1, FEETIER)
        df_positions_fees.loc[index, 'fees'] = fees
        
    return df_positions_fees

### Main

In [11]:
### input: money USD, upper and lower
# maths:
# p(i) = 1.0001**i , where i = tick
# i = math.floor(math.log(sqrt(P)/(2**96), math.sqrt(1.0001)))

# of positions:
# liquidity L = sqrt(token0_amt*token1__amt) ?? > may need the complicated 3 cases maths
# - Fee income = (L_you/L_pool) * swap volume under fixed time period (USD) * pool_fee_rate/100

In [12]:
# STEP 0:  PROVIDE BY USERS
UPPER = 1/2017.9452 # token1 per token0 (in terms of price)
LOWER = 1/2365.8978 # 1/2376.6075 
FEETIER = '500' # 0.3 = 3000/10000
TOKEN0 = 'DAI'
TOKEN1 = 'WETH'
MINT_TIME = 1627171200 # needs to start with the hour (unixTime % 3600 = 0) 
BURN_TIME = 1627344000
# 25 June 0000 - 27 July 0000 (GMT+0)

In [13]:
# Get required constants
CPRICE = get_cprice_at_mint(TOKEN0, TOKEN1, FEETIER, MINT_TIME)
DECIMAL0 = get_decimals(TOKEN0)
DECIMAL1 = get_decimals(TOKEN1)
print(CPRICE, 1/CPRICE)
print(DECIMAL0, DECIMAL1)

 
get_token_id: DAI
{'data': {'tokens': [{'id': '0x6b175474e89094c44da98b954eedeac495271d0f', 'name': 'Dai Stablecoin', 'symbol': 'DAI'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 500
{'data': {'pools': [{'feeTier': '500', 'id': '0x60594a405d53811d3bc4766596efd80fd545a270', 'token0': {'symbol': 'DAI'}, 'token1': {'symbol': 'WETH'}}]}}
0.0004650729232047768 2150.2004311691326
18.0 18.0


In [14]:
# STEP 1 (TODO): CALCULATED BY ALGORITHM, JNP's amounts_relation ?
# THIS CHANGES over time when swaps happens, and we need this to calculate IL
MINT_AMT0 = 4174.48 
MINT_AMT1 = 2.53

In [15]:
# STEP 3 (TODO): Generate df_positions
df_positions = get_positions_fixed_strategy(MINT_TIME, BURN_TIME, UPPER, LOWER)

In [16]:
# STEP 4 (TODO): Calculate Fees, IL and PNL at each hour, and append to df_positions
df_merged = pd.read_csv('../data/df_merged_DAI-WETH-3000-timestamp-1627466400-1620158400.csv')
df_positions_fees = calculate_pnl_from_positions(df_merged, df_positions)
df_positions_fees['fees'].sum()



In [22]:
# STEP 5 (TODO): also accoutning for minting/burning/swaping costs into PNL

In [31]:
df_positions_fees

,index,periodStartUnix,upper,lower,upper_tick,lower_tick,tick,liquidity,amountUSD,fees
0,0,1627171200,0.000496,0.000423,-76103,-77694,-76742.0,5.803192e+23,2.296948e+05,0.499611
1,1,1627174800,0.000496,0.000423,-76103,-77694,-76721.0,5.786429e+23,4.509750e+04,0.096202
2,2,1627178400,0.000496,0.000423,-76103,-77694,-76776.0,5.803192e+23,7.492462e+04,0.169150
3,3,1627182000,0.000496,0.000423,-76103,-77694,-76778.0,5.798590e+23,5.265855e+04,0.119242
4,4,1627185600,0.000496,0.000423,-76103,-77694,-76797.0,5.797656e+23,1.031571e+05,0.234143
5,5,1627189200,0.000496,0.000423,-76103,-77694,-76777.0,5.797668e+23,6.824075e+04,0.154379
6,6,1627192800,0.000496,0.000423,-76103,-77694,-76798.0,5.794441e+23,2.809074e+04,0.063702
7,7,1627196400,0.000496,0.000423,-76103,-77694,-76866.0,5.806810e+23,9.113653e+04,0.187530
8,8,1627200000,0.000496,0.000423,-76103,-77694,-76819.0,5.799830e+23,7.343628e+04,0.161412
9,9,1627203600,0.000496,0.000423,-76103,-77694,NaN,NaN,NaN,0.000000


In [ ]:
df_merged = pd.read_csv('../data/df_merged_DAI-WETH-3000-timestamp-1627466400-1620158400.csv')
watch_list = ['periodStartUnix',  
              'txCount', 'swaps_txCount', # check data integrity
              'amount0', 'amount1', 'amountUSD', # swaps data, note: amountUSD is sqrt(P)
              'tick', 'liquidity', 'sqrtPrice', 'tvlUSD', # pool data at that hour (liquidity is for the whole pool)
              'pool.token0.symbol', 'pool.token1.symbol', # token data
              'token0Price', 'token1Price'
             ]
# https://github.com/Uniswap/uniswap-v3-subgraph/blob/main/schema.graphql
df_merged[watch_list].sample(15)